Reranking Hybrid Statergy Implementation

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser

In [2]:
# Load the text file
loader = TextLoader('sample.txt')
raw_docs = loader.load()
raw_docs

[Document(metadata={'source': 'sample.txt'}, page_content='LangChain: LangChain serves as an extensive open-source orchestration framework that simplifies the creation of complex, LLM-powered applications by providing a modular suite of tools for prompt management and document loading while abstracting the intricacies of model integration to allow for seamless switching between different APIs with minimal code changes.\nLangGraph: LangGraph is a specialized library within the LangChain ecosystem that introduces a robust, graph-based architecture to enable stateful, multi-agent applications that move beyond linear paths by representing workflows as a series of nodes and edges for granular control over execution.\nFAISS: FAISS is a high-performance, open-source library developed by Metaâ€™s AI research team that specializes in the efficient similarity search and clustering of dense vectors at a massive scale, utilizing advanced indexing techniques like k-means clustering and product quan

In [3]:
#Split the text into document chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)
docs

[Document(metadata={'source': 'sample.txt'}, page_content='LangChain: LangChain serves as an extensive open-source orchestration framework that simplifies the creation of complex, LLM-powered applications by providing a modular suite of tools for prompt management and document loading while abstracting the intricacies of model integration to allow for seamless switching between different APIs with minimal code changes.'),
 Document(metadata={'source': 'sample.txt'}, page_content='LangGraph: LangGraph is a specialized library within the LangChain ecosystem that introduces a robust, graph-based architecture to enable stateful, multi-agent applications that move beyond linear paths by representing workflows as a series of nodes and edges for granular control over execution.'),
 Document(metadata={'source': 'sample.txt'}, page_content='FAISS: FAISS is a high-performance, open-source library developed by Metaâ€™s AI research team that specializes in the efficient similarity search and clust

In [4]:
# User query
query = 'How to use langchain to build memory application?'

In [5]:
# FAISS anf HuggingFace model Embeddings

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vectorstore = FAISS.from_documents(docs, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={'k': 8})

In [7]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000185E683D850>, search_kwargs={'k': 8})

In [8]:
# Using OPENAI
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

In [9]:
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key is None:
#     raise ValueError('OPENAI_API_KEY is not set')

# print(f'API Key loaded: {api_key[:5]} + .........')

In [10]:
embedding_openai = OpenAIEmbeddings()
vectorstore_openai = FAISS.from_documents(docs, embedding_openai)
retriever_openai = vectorstore_openai.as_retriever(search_kwargs={'k': 8})

In [11]:
retriever_openai

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000185ECA27D90>, search_kwargs={'k': 8})

In [12]:
from langchain.chat_models import init_chat_model


In [13]:
# Define prompt and use llm

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = init_chat_model('groq:llama-3.1-8b-instant')

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000185ED773450>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000185ECFC7010>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
# Prompt Template

# Define the template for how the LLM should process the documents
prompt = PromptTemplate.from_template(
    """
    You are a helpful assistant. Your task is to rank the following documents from most to least relevant.
    
    User Question: '{question}'
    
    documents: {documents}
        -Think about the relevance of each document to the user's question.
        -Return a list of document indices in ranked order, starting from the most relevant.
        
    Output format: comma separated document indices (e.g. , 2, 3, 0,...)
    
    """
)

In [15]:
#Create chain

# Build the LCEL chain (Prompt -> Model -> String Output)
chain = prompt | llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template="\n    You are a helpful assistant. Your task is to rank the following documents from most to least relevant.\n\n    User Question: '{question}'\n\n    documents: {documents}\n        -Think about the relevance of each document to the user's question.\n        -Return a list of document indices in ranked order, starting from the most relevant.\n\n    Output format: comma separated document indices (e.g. , 2, 3, 0,...)\n\n    ")
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000185ED773450>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000185ECFC7010>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [16]:
# Fetch relevant documents using your existing retriever
retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(id='1da02bab-954f-48a5-8bdc-80f158667d72', metadata={'source': 'sample.txt'}, page_content='Memory: Memory in LangChain refers to the dedicated module responsible for storing and retrieving information from past interactions, which allows conversational AI to maintain context and remember user preferences over multiple turns rather than treating each prompt as an isolated event.'),
 Document(id='1ba3abfd-29f4-4df6-bd0e-7052efc00a1f', metadata={'source': 'sample.txt'}, page_content='LangChain: LangChain serves as an extensive open-source orchestration framework that simplifies the creation of complex, LLM-powered applications by providing a modular suite of tools for prompt management and document loading while abstracting the intricacies of model integration to allow for seamless switching between different APIs with minimal code changes.'),
 Document(id='b55cf1d9-3d8e-4f06-b0af-2b7eb34230fb', metadata={'source': 'sample.txt'}, page_content='LangChain: By centering its core p

In [17]:
# Create a numbered list of the document contents
doc_lines = [
    f'{i + 1}. {doc.page_content}' for i, doc in enumerate(retrieved_docs)
]

# Join the lines into a single string for the prompt
formatted_docs = '\n'.join(doc_lines)
print(doc_lines)
print('')
print(formatted_docs)

['1. Memory: Memory in LangChain refers to the dedicated module responsible for storing and retrieving information from past interactions, which allows conversational AI to maintain context and remember user preferences over multiple turns rather than treating each prompt as an isolated event.', '2. LangChain: LangChain serves as an extensive open-source orchestration framework that simplifies the creation of complex, LLM-powered applications by providing a modular suite of tools for prompt management and document loading while abstracting the intricacies of model integration to allow for seamless switching between different APIs with minimal code changes.', '3. LangChain: By centering its core philosophy on the "Chain" concept, the framework enables the automated linking of various componentsâ€”such as memory, external tools, and promptsâ€”into a unified workflow designed to handle multi-step reasoning tasks while building robust AI systems that maintain data connectivity across diffe

In [18]:
# Reranking
#Invoke the chain with the variables
response = chain.invoke({
    'question': query,
    'documents': formatted_docs,
})

# 3. View the final ranked list
print("Re-ranked Indices:")
print(response)

Re-ranked Indices:
Based on the user question 'How to use langchain to build memory application?', I would rank the documents as follows:

3, 1, 4, 2, 5, 6, 8, 7

Here's a brief explanation for each ranking:

1. Document 3: This document directly mentions the concept of "Chain" in LangChain, which is closely related to building memory applications. It mentions the automated linking of various components, including memory, which is exactly what the user is asking for.

2. Document 1: This document specifically explains the concept of Memory in LangChain, which is a key component in building memory applications. It provides a clear understanding of how LangChain's memory module works.

3. Document 4: This document describes the modular architecture of LangChain, which enables developers to construct sophisticated AI pipelines. While it doesn't directly mention memory applications, it provides a general understanding of how LangChain works, which is relevant to the user's question.

4. Do

In [19]:
indices = []                          # Empty list

for i in response.split(','):         # Loop over items split by comma
    cleaned = i.strip()               # Remove spaces
    if cleaned.isdigit():             # Check if digits only
        number = int(cleaned) - 1     # Convert to int, subtract 1
        indices.append(number)        # Add to list

indices                               # Show result


[0, 3, 1, 4, 5, 7]

In [20]:
# Parse and rerank

# indices = [int(i.strip()) - 1 for x in response.split(',') if i.strip().isdigit()]
# indices
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]

reranked_docs

[Document(id='1da02bab-954f-48a5-8bdc-80f158667d72', metadata={'source': 'sample.txt'}, page_content='Memory: Memory in LangChain refers to the dedicated module responsible for storing and retrieving information from past interactions, which allows conversational AI to maintain context and remember user preferences over multiple turns rather than treating each prompt as an isolated event.'),
 Document(id='be62dada-ec30-4ff5-99e9-6923658bb09e', metadata={'source': 'sample.txt'}, page_content='LangChain: This modular architecture empowers developers to construct sophisticated AI pipelines where various document loaders and sequence chains work in harmony, ensuring that the transition between diverse language models remains fluid and the overall system stays resilient to changes in the underlying technology stack.'),
 Document(id='1ba3abfd-29f4-4df6-bd0e-7052efc00a1f', metadata={'source': 'sample.txt'}, page_content='LangChain: LangChain serves as an extensive open-source orchestration fr

In [21]:
for i, doc in enumerate(reranked_docs, 1):
    print(f'\nRank {i}:\n{doc.page_content}')


Rank 1:
Memory: Memory in LangChain refers to the dedicated module responsible for storing and retrieving information from past interactions, which allows conversational AI to maintain context and remember user preferences over multiple turns rather than treating each prompt as an isolated event.

Rank 2:
LangChain: This modular architecture empowers developers to construct sophisticated AI pipelines where various document loaders and sequence chains work in harmony, ensuring that the transition between diverse language models remains fluid and the overall system stays resilient to changes in the underlying technology stack.

Rank 3:
LangChain: LangChain serves as an extensive open-source orchestration framework that simplifies the creation of complex, LLM-powered applications by providing a modular suite of tools for prompt management and document loading while abstracting the intricacies of model integration to allow for seamless switching between different APIs with minimal code ch